In [16]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2

In [17]:
sales_data = pd.read_csv('sales_data.csv')

In [18]:
sales_data.head()

,Order_ID,Branch,Order_Date,Order_Priority,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,...,GPU,OS,OS_Version,Weight,Price,Quantity,Discount,Total_Price,Profit,Ship_Duration
0,0,Hamedan,1396-10-26,H,Dell,Vostro 3568,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,...,AMD Radeon R5 M420,Windows,10,2.18kg,3450000,1,0,3450000.0,250000,3
1,1,Kermanshah,1394-04-21,H,Lenovo,Yoga 720-13IKB,2 in 1 Convertible,"13.3""",IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,...,Intel HD Graphics 620,Windows,10,1.3kg,3400000,1,0,3400000.0,380000,2
2,2,Rasht,1402-05-24,M,Lenovo,IdeaPad Y700-15ISK,Notebook,"15.6""",IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,...,Nvidia GeForce GTX 960M,Windows,10,2.6kg,44900000,1,0,44900000.0,5630000,5
3,3,Kermanshah,1397-06-17,H,Dell,Inspiron 5370,Ultrabook,"13.3""",IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,...,AMD Radeon 530,Windows,10,1.4kg,12550000,1,0,12550000.0,1130000,3
4,4,Karaj,1392-09-30,L,Acer,Aspire E5-576G,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,...,Nvidia GeForce 940MX,Windows,10,2.23kg,2050000,1,0,2050000.0,180000,8


In [19]:
sales_data['Order_Date'] = sales_data['Order_Date'].apply(lambda x: int(''.join(x.split('-'))))
sales_data['Screen_Size'] = sales_data['Screen_Size'].apply(lambda x: float(x.split('"')[0]))
sales_data['RAM'] = sales_data['RAM'].apply(lambda x: int(x.split('GB')[0]))
sales_data['Weight'] = sales_data['Weight'].apply(lambda x: float(x.split('kg')[0]))

In [20]:
sales_data['Screen_Size'] = sales_data['Screen_Size'].apply(lambda x: int(x*10))
sales_data['Weight'] = sales_data['Weight'].apply(lambda x: int(x*100))
sales_data['Total_Price'] = sales_data['Total_Price'].apply(lambda x: int(x*10))

In [21]:
sales_data.drop(columns=['Order_ID', 'Branch', 'Order_Date', 'Order_Priority', 'Quantity', 'Total_Price'], inplace=True)

In [22]:
sales_data.columns

Index(['Manufacturer', 'Model_Name', 'Category', 'Screen_Size', 'Screen',
       'CPU', 'RAM', 'Storage', 'GPU', 'OS', 'OS_Version', 'Weight', 'Price',
       'Discount', 'Profit', 'Ship_Duration'],
      dtype='object')

In [23]:
sales_data.head()

,Manufacturer,Model_Name,Category,Screen_Size,Screen,CPU,RAM,Storage,GPU,OS,OS_Version,Weight,Price,Discount,Profit,Ship_Duration
0,Dell,Vostro 3568,Notebook,156,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,AMD Radeon R5 M420,Windows,10,218,3450000,0,250000,3
1,Lenovo,Yoga 720-13IKB,2 in 1 Convertible,133,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows,10,130,3400000,0,380000,2
2,Lenovo,IdeaPad Y700-15ISK,Notebook,156,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8,1TB HDD,Nvidia GeForce GTX 960M,Windows,10,260,44900000,0,5630000,5
3,Dell,Inspiron 5370,Ultrabook,133,IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8,256GB SSD,AMD Radeon 530,Windows,10,140,12550000,0,1130000,3
4,Acer,Aspire E5-576G,Notebook,156,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,256GB SSD,Nvidia GeForce 940MX,Windows,10,223,2050000,0,180000,8


In [24]:
X_encoded = sales_data.copy()

label_encoders = {}
categorical_cols = ['Manufacturer', 'Model_Name', 'Category', 'Screen', 'CPU', 'Storage', 'GPU', 'OS', 'OS_Version']
for col in categorical_cols:
    le = LabelEncoder()
    X_encoded[col + "_en"] = le.fit_transform(X_encoded[col])
    label_encoders[col] = le

X_encoded.drop(columns=['Manufacturer', 'Model_Name', 'Category', 'Screen', 'CPU', 'Storage', 'GPU', 'OS', 'OS_Version'],inplace=True)

In [25]:
X_encoded

,Screen_Size,RAM,Weight,Price,Discount,Profit,Ship_Duration,Manufacturer_en,Model_Name_en,Category_en,Screen_en,CPU_en,Storage_en,GPU_en,OS_en,OS_Version_en
0,156,8,218,3450000,0,250000,3,4,547,3,8,74,15,22,5,0
1,133,8,130,3400000,0,380000,2,10,585,0,13,74,15,53,5,0
2,156,8,260,44900000,0,5630000,5,10,225,3,15,92,8,89,5,0
3,133,8,140,12550000,0,1130000,3,4,242,4,15,106,15,7,5,0
4,156,4,223,2050000,0,180000,8,0,71,3,8,74,15,72,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017076,156,4,240,3800000,0,460000,4,10,291,1,15,102,8,75,5,0
1017077,156,4,204,1300000,0,120000,6,10,529,3,8,83,25,53,5,0
1017078,156,8,198,16700000,0,1700000,2,4,551,3,8,74,8,53,2,4
1017079,133,8,134,3750000,0,420000,10,7,121,4,21,97,15,53,5,0


In [26]:
y = X_encoded['Profit']
X = X_encoded.drop(columns=['Profit'])

In [12]:
selector = VarianceThreshold(threshold=0.4)
X_new = selector.fit_transform(X)
mask = selector.get_support()
selected_feat = X[mask]
sorted_feat = selected_feat[np.argsort(selector.variances_[mask])]

print(sorted_feat)


ValueError: Item wrong length 15 instead of 1017081.

In [13]:
selector = SelectKBest(score_func=chi2, k=1)
X.drop(columns=["Price"], inplace=True)
X_new = selector.fit_transform(X, y)

In [14]:
X_new

array([[547],
       [585],
       [225],
       ...,
       [551],
       [121],
       [239]])

In [15]:
for i in X_encoded.columns:
    if X_encoded[i][0] == X_new[0][0]:
        print(i)
# X_new

Model_Name_en


In [275]:
X_new

array([[156,   8, 218, ...,  22,   5,   0],
       [133,   8, 130, ...,  53,   5,   0],
       [156,   8, 260, ...,  89,   5,   0],
       ...,
       [156,   8, 198, ...,  53,   2,   4],
       [133,   8, 134, ...,  53,   5,   0],
       [156,   4, 229, ...,  24,   5,   0]])

In [27]:
model = RandomForestRegressor() 
model.fit(X, y)  
importance = model.feature_importances_
indices = np.argsort(importance)[::-1]

In [28]:
indices

array([ 3,  5,  7,  2, 10, 12, 11,  4,  9,  6,  0,  1,  8, 14, 13])

In [29]:
ni = -1
for i in range(len(indices)):
    ni += 1
    if indices[i] == min(indices):
        most_eff_ind = ni

In [30]:
X_encoded.iloc[:,indices[most_eff_ind]]

0          156
1          133
2          156
3          133
4          156
          ... 
1017076    156
1017077    156
1017078    156
1017079    133
1017080    156
Name: Screen_Size, Length: 1017081, dtype: int64

In [292]:
indices[10]

0